In [ ]:
pip install category_encoders

In [ ]:
import pandas as pd
from category_encoders import TargetEncoder
import sklearn as sk
from sklearn.utils import shuffle
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("train-data.csv", sep = ";")[:1000]
X_test = pd.read_csv("X_test.csv", sep = ";")[:1000]

#First, drop duplicates
train = train.drop_duplicates()
X_test = X_test.drop_duplicates()

print(f"Total dataset length is {len(train) + len(X_test)}.")

## Drop useless features

Some features don't add any information to the prediction of the 4th month of sale. So we're going to delete them.

In [12]:
train.drop(columns=['index', 'Cluster'], inplace=True)
X_test.drop(columns=['index', 'Cluster'], inplace=True)

## Merge extra datasets with the train set

#### Some useful variables used throughout the merges

In [13]:
dates = train['Date'].unique().tolist()

country_mapping = {
    'DE': 'Germany',
    'CN': 'China',
    'GB': 'United Kingdom',
    'AU': 'Australia',
    'ES': 'Spain',
    'NL': 'Netherlands',
    'US': 'United States',
    'DK': 'Denmark',
    'BE': 'Belgium',
    'FR': 'France',
    'IT': 'Italy',
    'PL': 'Poland',
    'SE': 'Sweden',
    'TW': 'Taiwan',
    'JP': 'Japan',
    'HK': 'Hong Kong',
    'KR': 'Korea, Rep.',
    'PH': 'Philippines',
    'MY': 'Malaysia',
    'SG': 'Singapore',
    'TH': 'Thailand',
    'Id': 'Indonesia',
    'FJ': 'Fiji',
    'VN': 'Vietnam',
    'BN': 'Brunei Darussalam',
    'NZ': 'New Zealand',
    'MM': 'Myanmar',
    'KH': 'Cambodia',
    'MN': 'Mongolia',
    'LA': 'Laos',
    'PG': 'Papua New Guinea'
}

inverted_country_mapping = {v: k for k, v in country_mapping.items()}

sites = train['Site'].unique().tolist()

### LPIExtend dataset

In [14]:
LPI = pd.read_csv('./extra-dataset/LPIextend.csv').drop(['Unnamed: 0', 'ID'], axis=1)

LPI['Country'] = LPI['Country'].apply(lambda x: 'Brunei Darussalam' if x == 'Brunei' else x)
LPI = LPI.loc[LPI['Country'].isin(country_mapping.values())]
rel_cols = ['Country', 'unMember', 'netChange', 'growthRate', 'worldPercentage', 'density', 'rank', 'Customs Score', 'Infrastructure Score', 'International Shipments Score', 'Logistics Competence and Quality Score', 'Timeliness Score', 'Tracking and Tracing Score']

LPI = LPI.loc[:, rel_cols]
LPI.at[133, 'unMember'] = True
for col in rel_cols[2:]:
    LPI[col].fillna(LPI[col].mean(), inplace=True)

LPI = LPI.reset_index(drop=True)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

subset_df = LPI[rel_cols[1:]]

scaler = StandardScaler()
subset_df_scaled = scaler.fit_transform(subset_df)

pca = PCA(n_components=2)
pca_result = pca.fit_transform(subset_df_scaled)

LPI['Country'] = LPI['Country'].apply(lambda x: inverted_country_mapping[x])

pca_df = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(2)])
final_lpi = pd.concat([LPI['Country'], pca_df['PC1'], pca_df['PC2']], axis=1)

train = pd.merge(train, final_lpi, how = 'left', on = "Country")
X_test = pd.merge(X_test, final_lpi, how = 'left', on = "Country")

#### Worldbank economic dataset

In [17]:
worldbank_eco = pd.read_csv("./extra-dataset/worldbank_economic_data.csv")
worldbank_eco.columns

worldbank_eco = pd.read_csv("./extra-dataset/worldbank_economic_data.csv")

#Rename columns so that its actually manipulable 
features_name = {'Exports of goods and services (annual % growth)' : 'Exports',
                 'GDP (current US$)' : 'GDP',
                 'Imports of goods and services (annual % growth)' : 'Imports',
                 'Industry (including construction), value added (annual % growth)': 'Industry',
                 'Manufacturing, value added (annual % growth)' : 'Manufacturing',
                 'Services, value added (annual % growth)': 'Services'}
                 
worldbank_eco.rename(columns=features_name, inplace=True)

#Keeping the columns that we considered the most interesting, and the keys.
worldbank_eco = worldbank_eco[['Country', 'Year', 'Exports', 'Imports', 'GDP', 'Industry', 'Manufacturing','Services']]

#Converting dates to the time spans of the train dataset
worldbank_eco['Date'] = 0

for index, row in worldbank_eco.iterrows():
    year = row['Year']
    country = worldbank_eco.loc[index,'Country']

    if country in country_mapping.values():
        worldbank_eco.loc[index,'Country'] = inverted_country_mapping[country]
    else:
        worldbank_eco = worldbank_eco.drop(index)
        pass

    for date in dates:
        if str(year) in date:
            worldbank_eco.loc[index, 'Date'] = date
            worldbank_eco.loc[len(worldbank_eco)] = worldbank_eco.loc[index]

worldbank_eco = worldbank_eco.drop_duplicates()

#NA imputing
impute_columns = []
for column_to_impute in ['Exports', 'Imports', 'GDP', 'Industry', 'Manufacturing','Services']:
    # Step 1: Impute missing values with the average value per country
    worldbank_eco[column_to_impute] = worldbank_eco.groupby('Country')[column_to_impute].transform(lambda x: x.fillna(x.mean()))

    # Step 2: Impute missing values with the international average if the whole country is missing
    worldbank_eco[column_to_impute] = worldbank_eco[column_to_impute].fillna(worldbank_eco[column_to_impute].mean())

    jan_apr_2023_mask = worldbank_eco['Date'] == 'jan-apr 2023'
    may_aug_2022_mask = worldbank_eco['Date'] == 'may-aug 2022'

    # Extract values for "may-aug 2022"
    may_aug_2022_values = worldbank_eco.loc[may_aug_2022_mask, ['Country', column_to_impute]]

    # Merge values for "may-aug 2022" into the main DataFrame for "jan-apr 2023"
    worldbank_eco.loc[jan_apr_2023_mask, column_to_impute] = worldbank_eco.loc[jan_apr_2023_mask, 'Country'].map(
        may_aug_2022_values.set_index('Country')[column_to_impute]
    )
    
#Merge dataset
train = pd.merge(train, worldbank_eco, on=['Country', 'Date'], how='left')
X_test = pd.merge(X_test, worldbank_eco, on=['Country', 'Date'], how='left')
train.pop('Year')
X_test.pop('Year')

for column_to_impute in ['Exports', 'Imports', 'GDP', 'Industry', 'Manufacturing','Services']:
    train[column_to_impute].fillna(train[column_to_impute].mean(), inplace = True)
    X_test[column_to_impute].fillna(X_test[column_to_impute].mean(), inplace = True)

train.head(10)

,id_product,Region,Country,Site,Operations,Zone,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,...,Month 4,PC1,PC2,Year,Exports,Imports,GDP,Industry,Manufacturing,Services
0,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,...,0,-2.499656,-0.765950,2021.0,9.710000,9.000000,4.259935e+12,3.61,5.070000,2.27
1,224631,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,...,0,-1.095021,-3.284178,2021.0,5.127357,7.232429,1.782046e+13,8.67,3.032473,8.49
2,104047,EUROPE,GB,NL_DC_HLD,Europe Operations,UK and Ireland,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,...,0,-0.641699,-0.701010,2021.0,2.240000,6.190000,3.122480e+12,9.29,10.080000,7.53
3,11823,EAJP,AU,AU_DC_Perth,International Operations,Pacific,reference-672,Product Line-1,Division-1,Customer Segmentation-11,...,0,-0.512168,-0.162840,2023.0,-0.100000,7.080000,1.675419e+12,1.67,2.540000,4.28
4,159924,EUROPE,ES,NL_DC_HLD,Europe Operations,Iberia,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,...,0,-1.712218,-0.551927,2021.0,14.380000,13.920000,1.427381e+12,4.03,8.910000,5.95
5,149445,EUROPE,ES,ES_DC_IberiaSantBoi,Europe Operations,Iberia,reference-7529,Product Line-4,Division-3,Customer Segmentation-3,...,0,-1.712218,-0.551927,2020.0,-19.940000,-14.850000,1.276963e+12,-13.14,-15.390000,-11.45
6,25324,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,reference-1882,Product Line-1,Division-1,Customer Segmentation-7,...,0,-0.512168,-0.162840,2022.0,-0.100000,7.080000,1.675419e+12,1.67,2.540000,4.28
7,168061,EUROPE,NL,NL_DC_Venray,Europe Operations,BeNe,reference-10764,Product Line-4,Division-3,Customer Segmentation-15,...,0,-2.128553,0.118574,2020.0,-4.330000,-4.750000,9.097935e+11,-2.09,-2.260000,-4.68
8,64939,NAM,US,US_DC_Mechanicsburg,North America Operations,US,reference-6848,Product Line-4,Division-3,Customer Segmentation-4,...,0,-0.963586,-2.246959,2021.0,6.050000,14.130000,2.331508e+13,3.25,6.790000,6.63
9,140385,EUROPE,DE,DE_FO_BNDch,Europe Operations,DACH,reference-6404,Product Line-4,Division-3,Customer Segmentation-1,...,0,-2.499656,-0.765950,2022.0,2.870000,5.990000,4.072192e+12,-0.47,0.200000,2.81


#### Worldbank inflation dataset

In [18]:
wbi = pd.read_csv('./extra-dataset/worldbank_inflation_data.csv')

transformation = {'Lao, PDR': 'Laos', 'Taiwan, China': 'Taiwan', 'Hong Kong SAR, China': 'Hong Kong'}
wbi['Country'] = wbi['Country'].apply(lambda x: transformation[x] if x in transformation.keys() else x)

 # Apply country mapping and keep only the associated data
train['Country'] = train['Country'].apply(lambda x: country_mapping[x])
X_test['Country'] = X_test['Country'].apply(lambda x: country_mapping[x])
wbi = wbi.loc[wbi['Country'].isin(country_mapping.values())]

 # Extract the date info
wbi['Year'] = wbi['Year-Month'].apply(lambda x: int(x[:4]))
wbi['Month'] = wbi['Year-Month'].apply(lambda x: int(x[-2:]))

 # Drop data out of the relevant temporal window
cond = (wbi['Year'] == 2020) & (wbi['Month'] <= 9)
wbi.drop(wbi[cond].index, inplace=True)
cond2 = (wbi['Year'] == 2023) & (wbi['Month'] >= 8)
wbi.drop(wbi[cond2].index, inplace=True)

# Create the quarter variable for the incoming temporal average
wbi['Quarter'] = wbi['Year-Month'].apply(lambda x: (int(x[-2:]) - 1) // 4)


 # Temporal average of the indexes
gped = wbi.groupby(['Country', 'Year', 'Quarter'])[['Energy Price Index', "Headline Consumer Price Index"]].mean()

# Useful quantities (default value if there is a NaN)
month_map = {'jan': 0, 'may': 1, 'sep': 2}
mean_epi = wbi['Energy Price Index'].mean()
mean_hcpi = wbi['Headline Consumer Price Index'].mean()

def transfo_index(x, default_val, key):
    index = (x['Country'],
             int(x['Date'].split()[-1]),
             month_map[x['Date'].split()[0].split('-')[0]])
    
    if index in gped.index:
        return gped.loc[index][key]
    return default_val

# Apply final transformation and replace NaNs with global mean
def final_transfo(df):
    df['EPI'] = df.apply(lambda x: transfo_index(x, mean_epi, 'Energy Price Index'), axis=1)
    df['EPI'].fillna(mean_epi, inplace=True)

    df['HCPI'] = df.apply(lambda x: transfo_index(x, mean_hcpi, 'Headline Consumer Price Index'), axis=1)
    df['HCPI'].fillna(mean_hcpi, inplace=True)
    return df

train = final_transfo(train)
X_test = final_transfo(X_test)

#### GSCPI Dataset

In [19]:
gscpi = pd.read_csv("./extra-dataset/GSCPI_data.csv")

gscpi['Year-Month'] = pd.to_datetime(gscpi['Year-Month'])

# Define a function to map each date to its corresponding time span
def map_to_time_span(date):
    if date.month in [5, 6, 7, 8] and date.year == 2021:
        return 'may-aug 2021'
    elif date.month in [1, 2, 3, 4] and date.year == 2021:
        return 'jan-apr 2021'
    elif date.month in [1, 2, 3, 4] and date.year == 2023:
        return 'jan-apr 2023'
    elif date.month in [9, 10, 11, 12] and date.year == 2020:
        return 'sep-dec 2020'
    elif date.month in [1, 2, 3, 4] and date.year == 2022:
        return 'jan-apr 2022'
    elif date.month in [9, 10, 11, 12] and date.year == 2021:
        return 'sep-dec 2021'
    elif date.month in [9, 10, 11, 12] and date.year == 2022:
        return 'sep-dec 2022'
    elif date.month in [5, 6, 7, 8] and date.year == 2022:
        return 'may-aug 2022'
    elif date.month in [5, 6, 7] and date.year == 2023:
        return 'may-jul 2023'
    else:
        return None

# Map each date to its corresponding time span and create a new column in gscpi
gscpi['Time Span'] = gscpi['Year-Month'].apply(map_to_time_span)

# Drop rows where Time Span is None
gscpi = gscpi.dropna(subset=['Time Span'])

# Compute the average gscpi_val for each time span
average_gscpi = gscpi.groupby('Time Span')['GSCPI'].mean().reset_index()

# Merge the average_gscpi dataframe with the train dataframe
train = pd.merge(train, average_gscpi, how='left', left_on='Date', right_on='Time Span')
X_test = pd.merge(X_test, average_gscpi, how='left', left_on='Date', right_on='Time Span')

# Drop unnecessary columns
train = train.drop(['Time Span'], axis=1)
X_test = X_test.drop(['Time Span'], axis=1)

# Display the updated train dataframe
train.head(10)

,id_product,Region,Country,Site,Operations,Zone,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,...,Year,Exports,Imports,GDP,Industry,Manufacturing,Services,EPI,HCPI,GSCPI
0,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,...,2021.0,9.710000,9.000000,4.259935e+12,3.61,5.070000,2.27,108.703500,108.696750,2.957795
1,224631,CHINA,China,CN_DC_Shanghai,China Operations,China & HK,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,...,2021.0,5.127357,7.232429,1.782046e+13,8.67,3.032473,8.49,100.875000,129.505090,2.030159
2,104047,EUROPE,United Kingdom,NL_DC_HLD,Europe Operations,UK and Ireland,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,...,2021.0,2.240000,6.190000,3.122480e+12,9.29,10.080000,7.53,106.800000,109.700000,2.030159
3,11823,EAJP,Australia,AU_DC_Perth,International Operations,Pacific,reference-672,Product Line-1,Division-1,Customer Segmentation-11,...,2023.0,-0.100000,7.080000,1.675419e+12,1.67,2.540000,4.28,122.997285,126.769292,-0.430277
4,159924,EUROPE,Spain,NL_DC_HLD,Europe Operations,Iberia,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,...,2021.0,14.380000,13.920000,1.427381e+12,4.03,8.910000,5.95,101.715300,104.850000,2.030159
5,149445,EUROPE,Spain,ES_DC_IberiaSantBoi,Europe Operations,Iberia,reference-7529,Product Line-4,Division-3,Customer Segmentation-3,...,2020.0,-19.940000,-14.850000,1.276963e+12,-13.14,-15.390000,-11.45,93.161100,104.266667,0.784687
6,25324,EAJP,Australia,AU_DC_Sydney,International Operations,Pacific,reference-1882,Product Line-1,Division-1,Customer Segmentation-7,...,2022.0,-0.100000,7.080000,1.675419e+12,1.67,2.540000,4.28,122.997285,126.769292,3.117173
7,168061,EUROPE,Netherlands,NL_DC_Venray,Europe Operations,BeNe,reference-10764,Product Line-4,Division-3,Customer Segmentation-15,...,2020.0,-4.330000,-4.750000,9.097935e+11,-2.09,-2.260000,-4.68,102.373333,107.966667,0.784687
8,64939,NAM,United States,US_DC_Mechanicsburg,North America Operations,US,reference-6848,Product Line-4,Division-3,Customer Segmentation-4,...,2021.0,6.050000,14.130000,2.331508e+13,3.25,6.790000,6.63,125.541750,116.825000,3.897540
9,140385,EUROPE,Germany,DE_FO_BNDch,Europe Operations,DACH,reference-6404,Product Line-4,Division-3,Customer Segmentation-1,...,2022.0,2.870000,5.990000,4.072192e+12,-0.47,0.200000,2.81,133.327750,112.834750,3.117173


## Convert Months to proper type and use linear regression to impute NA Month 1 values.

In [20]:
imputed_mask = train['Month 1'].isna()
train['M_1_Imputed'] = np.where(imputed_mask, 1, 0)

imputed_mask = X_test['Month 1'].isna()
X_test['M_1_Imputed'] = np.where(imputed_mask, 1, 0)

def month_impute_and_convert(df):
    
    months = [col for col in df.columns if "Month" in col]
    data_month = df[months].copy()
    for m in months:
        data_month[m] = data_month[m].str.replace(" ", "")
        data_month[m] = pd.to_numeric(data_month[m])
        
    if df.equals(train):
        X = data_month.drop(["Month 1", "Month 4"], axis=1)
    else:
        X = data_month.drop(["Month 1"], axis=1)
    y = data_month["Month 1"]

    X_train = data_month.dropna(subset=["Month 1"])[["Month 2", "Month 3"]]
    y_train = y[X_train.index]

    X_test = X[pd.isna(data_month["Month 1"])]
    y_test = y[X_test.index]

    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    vals = lin_reg.predict(X_test)

    df["Month 1"] = data_month["Month 1"].copy()
    df.loc[X_test.index, "Month 1"] = vals.copy()
    
    if df.equals(train):
        df.loc[:, ["Month 2", "Month 3", "Month 4"]] = data_month.loc[:, ["Month 2", "Month 3", "Month 4"]].astype(int)
    else: 
        df.loc[:, ["Month 2", "Month 3"]] = data_month.loc[:, ["Month 2", "Month 3"]].astype(int)
    return df
    
train = month_impute_and_convert(train)
X_test = month_impute_and_convert(X_test)

## Feature encoding

We're going to encode categorical features.

* All geographical features: We might replace all those features by longitude and latitude coordinates in order to reduce features and to establish actual euclidian distance between observations.

* Reference proxy: Since there are too many proxies (22406), we're not going to one hot encode it but rather use target encoding.

* Division proxy: Since there are only 3 divisions, we're going to one hot encode this feature.

* Product line proxy: Since there are only 6 different lines, we're going to one hot encode this feature.

* Customer Persona proxy: there are 72 different values. We're going to use target encoding.

* Strategic Product Family proxy: There are 19 different values. We can discuss on whether to use target or one hot encoding.

* Product life cycle: There are 4 different values including NaN which is actually the most proeminent. Therefore, we're going to impute the NaN as an "Active" class and we're going to one hot encode the feature.

* Date: There are 9 diferent time spans. We can encode them ordinally.

In [21]:
def preprocess_data(df, encoder=None):
    # Target Encoding
    tgt_encode_columns = ['id_product', 'Reference proxy', 'Customer Persona proxy', 'Strategic Product Family proxy']

    if encoder is None:
        encoder = TargetEncoder(cols=tgt_encode_columns)
        df_encoded = encoder.fit_transform(df, y_train)
    else:
        df_encoded = encoder.transform(df)

    # Update the original DataFrame with the encoded values
    df[tgt_encode_columns] = df_encoded[tgt_encode_columns]

    # Fill NA with "Unknown" Value for Product life cycle
    df['Product Life cycel status'].fillna('Active', inplace=True)

    # One hot encoding
    one_hot_encode_cols = ['Division proxy', 'Product  Line proxy', 'Product Life cycel status']
    df = pd.get_dummies(df, columns=one_hot_encode_cols)

    # Date
    date_set = {'may-aug 2021': 2, 'jan-apr 2021': 1, 'jan-apr 2023': 7, 'sep-dec 2020': 0,
                'jan-apr 2022': 4, 'sep-dec 2021': 3, 'sep-dec 2022': 6, 'may-aug 2022': 5, 'may-jul 2023': 8}
    df['Date'] = df['Date'].apply(lambda x: date_set[x])

    return df, encoder

y_train = train.pop('Month 4')
X_train = train.copy()
del(train)

X_train, encoder = preprocess_data(X_train)
X_test, _ = preprocess_data(X_test, encoder)

#### Replace geographical features by latitude & longitude + site type

In [22]:
#Import centers localisation and preprocess
centers_loc = pd.read_csv('distribution_centers.csv')
centers_loc.drop(columns = ['City', 'Country'], inplace = True)

#Import center types and preprocess
center_types = pd.read_csv('locations_combined.csv')
center_types.drop(columns = ['Country'], inplace = True)
center_types.rename(columns={'Location': 'Site'}, inplace=True)

def extract_between_parentheses(input_string):
    last_left_parenthesis = input_string.rfind('(')
    last_right_parenthesis = input_string.rfind(')')

    if last_left_parenthesis != -1 and last_right_parenthesis != -1:
        result = input_string[last_left_parenthesis + 1:last_right_parenthesis]
        return result
    else:
        return None

for index, row in center_types.iterrows():
    location = row['Site']
    center_types.loc[index, 'Site'] = extract_between_parentheses(location)

def insert_geo_features(df):
    df.drop(columns = ["Region", "Country", "Operations", "Zone"], inplace = True)
    df = pd.merge(df, center_types, on= 'Site', how = 'left')
    df = pd.merge(df, centers_loc, on = 'Site', how = 'left')
    df.pop('Site')
    return df
    
X_train = insert_geo_features(X_train)
X_test = insert_geo_features(X_test)

In [23]:
def convert_to_num(df):
    object_columns = df.select_dtypes(include=['object']).columns

    # Convert object columns to numeric
    df[object_columns] = df[object_columns].apply(pd.to_numeric)
    
    return df

X_train = convert_to_num(X_train)
X_test = convert_to_num(X_test)
y_train = y_train.apply(pd.to_numeric)